In [ ]:
# credits to KEVIN DWYER at http://blog.thehumangeo.com/2014/05/12/drawing-boundaries-in-python/ for this tutorial
# code for this entire notebook is copied and/or modified from his tutorial and is being used for learning and practice purposes
import fiona
import shapely.geometry as geometry
input_shapefile = 'simple_points.qgz'
shapefile = fiona.open(input_shapefile)
points = [geometry.shape(point['geometry'])
          for point in shapefile]



In [ ]:
import pylab as p1
x = [p.coords.xy[0] for p in points]
y = [p.coords.xy[1] for p in points]
p1.figure(figsize=(10,10))
_=p1.plot(x,y,'o', color='#f16824')

In [ ]:
point_collection = geometry.MultiPoint(list(points))
point_collection.envelope

In [ ]:
from descartes import PolygonPatch 
def plot_polygon(polygon):
    fig = p1.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    margin = 0.3
    x_min, y_min, x_max, y_max = polygon.bounds
    ax.set_xlim([x_min-margin, x_max+margin])
    ax.set_ylim([y_min-margin, y_max+margin])
    patch = PolygonPatch(polygon, fc='#999999',
                         ec='#000000', fill=True, 
                         zorder=-1)
    ax.add_patch(patch)
    return fig

_ = plot_polygon(point_collection.envelope)
_ = p1.plot(x,y,'o', color='#f16824')